###  Imports

In [ ]:
import torch

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

from dataset import FiftyOneTorchDataset
from model import create_model
from util import add_detections, get_transforms

import config
import pickle

torch.manual_seed(1)

In [ ]:
# dataset_name = "coco-2017-validation"
dataset_name = "ImageNet_validation"

# The directory containing the dataset to import
dataset_dir = "C:/Data_drive/Data/Imagenet/ImageNet/imagenet_val_dataset/imagenet_val_dataset"

In [ ]:
#fo.core.dataset.delete_non_persistent_datasets()

In [ ]:
# Check if the datasets exist on this machine
if fo.core.dataset.dataset_exists(dataset_name):

    fo_dataset = fo.load_dataset(dataset_name) # if the dataset  exists, load it
else:
    if dataset_name == "coco-2017-validation":
        fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")
    else: # if the dataset isnt coco, we will load it from the machine
        fo_dataset = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.VOCDetectionDataset,
        name=dataset_name,
        )



if dataset_name == "ImageNet_validation":
    with open('dataset_analysis/imagenet_dict_mapping.pkl', 'rb') as f:
                imagenet_class_mapping = pickle.load(f)
    fo_dataset = fo_dataset.map_labels("ground_truth", imagenet_class_mapping)


#needed to calculate image height and width
fo_dataset.compute_metadata()
#create the session to view the dataset
session = fo.launch_app(fo_dataset)

In [ ]:
# create the list of labels needed for evaluation, if evaluating on all labels, leave empty

known_unknowns = ['lizard', 'turtle', 'pen', 'cowboy hat', 'tank']
known_knowns = ['clock', 'vase', 'toaster', 'microwave', 'mouse', 'potted plant', 'sports ball', 'zebra', 'dog', 'bird', 'bench', 'parking meter', 'airplane', 'bicycle']



dataset_class_labels = known_knowns + known_unknowns

model_class_labels = known_knowns + known_unknowns


In [ ]:
#get the transformations needed for the images
_, test_transforms = get_transforms()

if len(dataset_class_labels) > 0:

    item_view = fo_dataset.filter_labels("ground_truth",
            F("label").is_in(dataset_class_labels))

    # find the class with the fewest examples
    class_count = item_view.count_values("ground_truth.detections.label")
    smallest_class = min(class_count, key=class_count.get) # find the key of the smallest class

    id = set() # create a set to contain the image ids

    for dataset_class in item_view.distinct("ground_truth.detections.label"): # loop through all of the class labels
        class_view = item_view.filter_labels("ground_truth",
            F("label").is_in(dataset_class)) # create a view from which to sample the class
        sample_ids = class_view.take(class_count[smallest_class], seed = 51) # take the number of classes based on the smallest class

        for sample in sample_ids:
            id.add(sample.id) # add the image ids to the set
    item_view = item_view.select(id) # create a view based on these images

    known_view = item_view.filter_labels("ground_truth",
            F("label").is_in(known_knowns))


    unknown_view = item_view.filter_labels("ground_truth",
            F("label").is_in(known_unknowns))


    # use our dataset and defined transformations
    known_evaluation_dataset = FiftyOneTorchDataset(known_view, test_transforms,
        classes=known_knowns)

    # use our dataset and defined transformations
    unknown_evaluation_dataset = FiftyOneTorchDataset(unknown_view, test_transforms,
        classes=known_unknowns)

    print(f'Evaluating on {len(known_evaluation_dataset)} known samples')
    print(f'Evaluating on {len(unknown_evaluation_dataset)} unknown samples')
else: # if we do not provide labels of interest
    item_view = fo_dataset

    #create an item list for use later
    dataset_class_labels = fo_dataset.distinct("ground_truth.detections.label")


print(f'Evaluating on {len(item_view)} samples')



# use our dataset and defined transformations
evaluation_dataset = FiftyOneTorchDataset(item_view, test_transforms,
        classes=dataset_class_labels)

session.view = item_view

# add a blank line dropped during classification
if model_class_labels[0] != 'background':
     model_class_labels.insert(0,'background')

In [ ]:
# for some items, CLIP may do better with different textual descriptions

replacements = {
    'mouse': 'computer mouse',
}

for k, v in replacements.items():
    CLIP_list = [v if item == k else item for item in model_class_labels]

print(CLIP_list)

In [ ]:
# a helper funciton to evaluate the model on certain datasets
def evaluate_model(model , field_name, evaluation_dataset, fiftyone_Dataset, classes = dataset_class_labels, BCC=False, iou=0.5, eps = 35, labelmap=dataset_class_labels):
    # model - the object detection model
    # field_name - the name used to add the model to FiftyOne
    # evaluation_dataset - the FiftyOne view on which to calculate the data
    # fiftyone_Dataset - the full FiftyOne dataset
    # Classes - the class list on which to evaluate the model
    # BCC - use Bounding box Clustering and Consolidation
    # iou - the iou threshold for nms
    # eps - the eps_neighborhood parameter from DBSCAN
    # labelmap - the mapping of model class number to dataset class number

    add_detections(model, evaluation_dataset, fiftyone_Dataset, field_name=field_name, PRED_CLUSTERING=BCC, labelmap = labelmap, eps = eps)

    evaluation = fo.evaluate_detections(
        item_view,
        field_name,
        classes=classes,
        eval_key=field_name,
        compute_mAP=True,
        iou=iou,
    )

    return evaluation

# Check CLIP RPN performance

In [ ]:
# test out the trained CLIP-FRCNN
MODEL_TYPE = 'CLIP-RPN'
WEIGHTS_NAME='CLIP-RPN_rpn_full_training epoch_30.pth'

# tokenize item list for CLIP
import clip
_, preprocess = clip.load("RN50", device=config.DEVICE)

# create the model
clip_frcnn_model = create_model(MODEL_TYPE, classes=CLIP_list)

# load the pre-trained model
checkpoint = torch.load(WEIGHTS_NAME)
clip_frcnn_model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']

print(f'loaded checkpoint at epoch {epoch}')

# set to evaluation mode
clip_frcnn_model.eval()

In [ ]:
# find performance without clustering

iou = .1
known_no_cluster_evaluation = evaluate_model(clip_frcnn_model,
                                       evaluation_dataset = known_evaluation_dataset,
                                       field_name = "clip_RPN_predictions_no_clustering",
                                       fiftyone_Dataset = fo_dataset,
                                       classes = known_knowns,
                                       BCC=False,
                                       iou=iou,
                                       labelmap=dataset_class_labels,
                                       )
unknown_no_cluster_evaluation = evaluate_model(clip_frcnn_model,
                                       evaluation_dataset = unknown_evaluation_dataset,
                                       field_name = "clip_RPN_predictions_no_clustering",
                                       fiftyone_Dataset = fo_dataset,
                                       classes = known_unknowns,
                                       BCC=False,
                                       iou=iou,
                                       labelmap=dataset_class_labels,
                                       )

print(f'Known mAP (no BCC): {known_no_cluster_evaluation.mAP()}')
known_no_cluster_evaluation.print_report()
print(f'Unknown mAP (no BCC): {unknown_no_cluster_evaluation.mAP()}')
unknown_no_cluster_evaluation.print_report()

In [ ]:
# find performance without clustering

iou = .5
known_no_cluster_evaluation = evaluate_model(clip_frcnn_model,
                                       evaluation_dataset = known_evaluation_dataset,
                                       field_name = "clip_RPN_predictions_no_clustering",
                                       fiftyone_Dataset = fo_dataset,
                                       classes = known_knowns,
                                       BCC=False,
                                       iou=iou,
                                       labelmap=dataset_class_labels,
                                       )
unknown_no_cluster_evaluation = evaluate_model(clip_frcnn_model,
                                       evaluation_dataset = unknown_evaluation_dataset,
                                       field_name = "clip_RPN_predictions_no_clustering",
                                       fiftyone_Dataset = fo_dataset,
                                       classes = known_unknowns,
                                       BCC=False,
                                       iou=iou,
                                       labelmap=dataset_class_labels,
                                       )

print(f'Known mAP (no BCC): {known_no_cluster_evaluation.mAP()}')
known_no_cluster_evaluation.print_report()
print(f'Unknown mAP (no BCC): {unknown_no_cluster_evaluation.mAP()}')
unknown_no_cluster_evaluation.print_report()

## Find performance with clustering

In [ ]:
eps = 35
iou = .1

BCC_evaluation = evaluate_model(clip_frcnn_model,
                                evaluation_dataset = evaluation_dataset,
                                field_name = f"clip_RPN_predictions_eps_{eps}",
                                fiftyone_Dataset = fo_dataset,
                                BCC=True,
                                iou=iou,
                                eps= eps
                                )

map = BCC_evaluation.mAP()
print(f'mAP = {map}')
BCC_evaluation.print_report()

In [ ]:
eps = 35
iou=0.5
BCC_evaluation = evaluate_model(clip_frcnn_model,
                                evaluation_dataset = evaluation_dataset,
                                field_name = f"clip_RPN_predictions_eps_{eps}",
                                fiftyone_Dataset = fo_dataset,
                                BCC=True,
                                iou=iou,
                                eps= eps
                                )

map = BCC_evaluation.mAP()
print(f'mAP = {map}')
BCC_evaluation.print_report()

In [ ]:
# Test on small objects only
# Bboxes are in [top-left-x, top-left-y, width, height] format
bbox_area = F("bounding_box")[2] * F("bounding_box")[3]

small_view = item_view.filter_labels("ground_truth", bbox_area < 0.2)
# use our dataset and defined transformations
small_object_dataset = FiftyOneTorchDataset(small_view, test_transforms,
        classes=dataset_class_labels)
iou=0.5
eps = 35
BCC_evaluation = evaluate_model(clip_frcnn_model,
                                evaluation_dataset = small_object_dataset,
                                field_name = f"clip_RPN_small_predictions_eps_{eps}",
                                fiftyone_Dataset = fo_dataset,
                                BCC=True,
                                iou=iou,
                                eps= eps
                                )

map = BCC_evaluation.mAP()
print(f'mAP = {map}')
BCC_evaluation.print_report()

In [ ]:
# Test on large objects only
# Bboxes are in [top-left-x, top-left-y, width, height] format
bbox_area = F("bounding_box")[2] * F("bounding_box")[3]

large_view = item_view.filter_labels("ground_truth", bbox_area > 0.2)
# use our dataset and defined transformations
large_object_dataset = FiftyOneTorchDataset(large_view, test_transforms,
        classes=dataset_class_labels)

iou=0.5
eps = 35

BCC_evaluation = evaluate_model(clip_frcnn_model,
                                evaluation_dataset = large_object_dataset,
                                field_name = f"clip_RPN_large_predictions_eps_{eps}",
                                fiftyone_Dataset = fo_dataset,
                                BCC=True,
                                iou=iou,
                                eps= eps
                                )

map = BCC_evaluation.mAP()
print(f'mAP = {map}')
BCC_evaluation.print_report()

#### Sensitivity Study

In [ ]:
# find performance with clustering

results = []
best_map = 0

for eps in range(5, 50, 5):
    print(f'epsilon = {eps}')

    evaluation = evaluate_model(clip_frcnn_model,
                                evaluation_dataset = evaluation_dataset,
                                field_name = f"clip_RPN_predictions_eps_{eps}",
                                fiftyone_Dataset = fo_dataset,
                                BCC=True,
                                iou=0.5,
                                eps= eps
                                )

    map = evaluation.mAP()
    print(f'mAP = {map}')

    results.append([eps, map])
    if map > best_map:
        best_eps = eps
        best_map = map
        best_eval = evaluation

print(f'best_mAP = {best_map}')
print(f'best_eps = {best_eps}')
best_eval.print_report()

In [ ]:
# importing the required module
import matplotlib.pyplot as plt
import pandas as pd
res = pd.DataFrame(results)
# plotting the points
plt.plot(res[0], res[1])

# naming the x axis
plt.xlabel('DBSCAN eps')
# naming the y axis
plt.ylabel('mean Average Precision')

# giving a title to my graph
plt.title('Box Clustering and Consolidation Sensitivity')

# function to show the plot

plt.savefig('Box Clustering and Consolidation Sensitivity.jpg')

# Test against Faster-RCNN

In [ ]:
# create the labelmap for the coco dataset
coco_labels = open("dataset_analysis/coco_labels.txt", "r")
coco_list = coco_labels.read().splitlines() # read each line in as a value in a list
coco_list.insert(0,'background') # add the background class
coco_id = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34,
          35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
          64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91] # annoyingly, COCO has 90 class ids but only 80 labels
coco = dict(zip(coco_id, coco_list)) # convert it to a dict

## Calculate metrics from the FRCNN using iou 0.1

In [ ]:
import torchvision
# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

iou=.1
pre_trained_known_evaluation = evaluate_model(model,
                                evaluation_dataset = known_evaluation_dataset,
                                field_name = "FRCNN_known_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_knowns,
                                iou=iou,
                                labelmap=coco
                                )
pre_trained_unknown_evaluation = evaluate_model(model,
                                evaluation_dataset = unknown_evaluation_dataset,
                                field_name = "FRCNN_unknown_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_unknowns,
                                iou=iou,
                                labelmap=coco
                                )

print(f'mAP: {pre_trained_known_evaluation.mAP()}')
pre_trained_known_evaluation.print_report()

print(f'mAP: {pre_trained_unknown_evaluation.mAP()}')
pre_trained_unknown_evaluation.print_report()

In [ ]:
iou=.5
pre_trained_known_evaluation = evaluate_model(model,
                                evaluation_dataset = known_evaluation_dataset,
                                field_name = "FRCNN_known_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_knowns,
                                iou=iou,
                                labelmap=coco
                                )
pre_trained_unknown_evaluation = evaluate_model(model,
                                evaluation_dataset = unknown_evaluation_dataset,
                                field_name = "FRCNN_unknown_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_unknowns,
                                iou=iou,
                                labelmap=coco
                                )

print(f'mAP: {pre_trained_known_evaluation.mAP()}')
pre_trained_known_evaluation.print_report()

print(f'mAP: {pre_trained_unknown_evaluation.mAP()}')
pre_trained_unknown_evaluation.print_report()

In [ ]:
plot = pre_trained_known_evaluation.plot_pr_curves(classes=known_knowns)
plot.show()

In [ ]:
plot2 = BCC_evaluation.plot_pr_curves(classes=known_unknowns)
plot2.show()

## Yolov5

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
iou=.5
pre_trained_known_evaluation = evaluate_model(model,
                                evaluation_dataset = known_evaluation_dataset,
                                field_name = "YOLO_known_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_knowns,
                                iou=iou,
                                labelmap=coco
                                )
pre_trained_unknown_evaluation = evaluate_model(model,
                                evaluation_dataset = unknown_evaluation_dataset,
                                field_name = "YOLO_unknown_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_unknowns,
                                iou=iou,
                                labelmap=coco
                                )

print(f'mAP: {pre_trained_known_evaluation.mAP()}')
pre_trained_known_evaluation.print_report()

print(f'mAP: {pre_trained_unknown_evaluation.mAP()}')
pre_trained_unknown_evaluation.print_report()

In [ ]:

session.view = item_view